# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# path
cities_path="../WeatherPy/Output/cities_weather.csv"
# Read the cities data
cities_data= pd.read_csv(cities_path)
cities_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Lincoln,-34.7333,135.8667,17.63,64,95,5.60,AU,2021-02-07 02:47:11
1,Altoona,40.5187,-78.3947,-5.56,68,83,0.84,US,2021-02-07 02:48:54
2,Bambous Virieux,-20.3428,57.7575,25.00,88,40,2.57,MU,2021-02-07 02:51:25
3,Buenaventura,3.8801,-77.0312,20.00,73,100,1.24,CO,2021-02-07 02:51:25
4,Matara,5.9485,80.5353,26.02,81,35,1.48,LK,2021-02-07 02:51:25
...,...,...,...,...,...,...,...,...,...
570,Cayenne,4.9333,-52.3333,25.00,100,75,1.54,GF,2021-02-07 02:52:53
571,Visby,57.6409,18.2960,-3.00,80,90,3.09,SE,2021-02-07 02:50:18
572,Calamar,10.2511,-74.9146,27.00,78,0,3.09,CO,2021-02-07 02:52:53
573,Doka,13.5167,35.7667,14.60,41,1,5.19,SD,2021-02-07 02:52:53


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#find the max humidity
city=[]
lat=[]
lng=[]
hum=[]
max =cities_data["Humidity"].max()
for index, row in cities_data.iterrows():
    if row["Humidity"] == max:
        city.append(row["City"])
        lat.append(row["Lat"])
        lng.append(row["Lng"])
        hum.append(row["Humidity"])
max_dic = {
    "City": city,
    "Lat": lat,
    "Lng": lng,
    "Humidity": hum
}
max_df = pd.DataFrame(max_dic)  
max_df
max_locations = max_df[["Lat", "Lng"]]

In [4]:
# Store latitude and longitude in locations
locations = cities_data[["Lat", "Lng"]]
# Fill NaN values and convert to float
humidity = cities_data["Humidity"].astype(float)
# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,max_intensity=30, point_radius=1)
max_heat= gmaps.symbol_layer(max_locations, fill_color='green', stroke_color='green', scale=3)
# Add layer
fig.add_layer(heat_layer)
fig.add_layer(max_heat)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
* Store into variable
* Add a "Hotel Name" column to the DataFrame.

In [29]:
hotel_df=cities_data.loc[(cities_data['Max Temp']>30)&(cities_data['Max Temp']<80)
                         &(cities_data['Wind Speed']<=10)&(cities_data['Cloudiness']<30)]
# Add columns for Country name, Hotel name, hotel rating
hotel_df=hotel_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Raiting"] = ""
hotel_df["Hotel Address"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Raiting,Hotel Address
0,Ambulu,-8.3450,113.6058,30.35,63,28,1.01,ID,2021-02-07 02:51:35,,,
1,Dawei,14.0833,98.2000,30.51,36,4,1.24,MM,2021-02-07 02:51:41,,,
2,Yulara,-25.2406,130.9889,31.00,22,0,7.20,AU,2021-02-07 02:51:03,,,
3,Alice Springs,-23.7000,133.8833,31.67,23,3,6.69,AU,2021-02-07 02:50:54,,,


### Hotel Map
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    name_address = requests.get(base_url, params=params)
    #print the name_address url, avoid doing for public github repos in order to avoid exposing key
    # convert to json
    name_address = name_address.json()
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Raiting"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index, "Hotel Name"] = np.nan
        hotel_df.loc[index, "Hotel Address"] = np.nan
        hotel_df.loc[index, "Hotel Raiting"] = np.nan

In [31]:
hotel_df=hotel_df.dropna(how='any')
hotel_df
# Save Data to csv
#cities_pd.to_csv("Airport_Output.csv")


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Raiting,Hotel Address
0,Ambulu,-8.3450,113.6058,30.35,63,28,1.01,ID,2021-02-07 02:51:35,Hotel Ambulu Jember,4.1,"Jl. Manggar No.200, Tegalsari, Kabupaten Jember"
1,Dawei,14.0833,98.2000,30.51,36,4,1.24,MM,2021-02-07 02:51:41,Hotel Dawei,4.3,"No.(7/A), Arzarni Rd Byaw Taw Wa Quarter"
2,Yulara,-25.2406,130.9889,31.00,22,0,7.20,AU,2021-02-07 02:51:03,Sails in the Desert,4.2,"163 Yulara Dr, Yulara"
3,Alice Springs,-23.7000,133.8833,31.67,23,3,6.69,AU,2021-02-07 02:50:54,DoubleTree by Hilton Hotel Alice Springs,4.1,"82 Barrett Dr, Alice Springs"


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [33]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
hotel_markers = gmaps.marker_layer(hotel_locations,info_box_content=hotel_info)
# Display figure
fig.add_layer(hotel_markers)
fig.add_layer (heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))